<a href="https://colab.research.google.com/github/HegdeSiddesh/cs6910_Assignment2/blob/main/Part%20B/Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CS6910 Assignment 2 Part B: 
The goal of this part is as follows:        
     
- Finetune a pre-trained model just as you would do in many real world applications              


### Import required packages

In [ ]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import math
from keras.layers import Conv2D , MaxPool2D , Flatten , Dropout, Dense, Activation, BatchNormalization, GlobalAveragePooling2D

import random
np.random.seed(137) # To ensure that the random number generated are the same for every iteration
import warnings
#warnings.filterwarnings("ignore")
!pip install --upgrade wandb
import wandb
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
import os
from keras.datasets import fashion_mnist
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Flatten, InputLayer
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation
from wandb.keras import WandbCallback
from google.colab.patches import cv2_imshow
from keras.callbacks import EarlyStopping
tf.random.set_seed(137)

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 144 kB 73.0 MB/s 
     |████████████████████████████████| 181 kB 54.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=9951be3c831e2f33753c0b337d0dc10cf00b12235bd71c289b2fab561b8bbade
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
wandb.login()

True

In [ ]:
#Download and unzip the iNaturalist dataset
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip -q /content/nature_12K.zip

--2022-04-01 09:56:43--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   202MB/s    in 17s     

2022-04-01 09:57:00 (220 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
wandb.init(project="Assignment_2", entity="hithesh-sidhesh", name="Part_B")

wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


###Create functions for creating models using pretrained models

In [ ]:
class ConfigValues():
    """
    Class to hold the parameters needed to create a model using the pretrained models.

    Attributes:
        layers_unfreeze:Number of layers from end of pretrained model to be used while training (default 20)
        model:The pretrained model to be used (default "InceptionResNetV2")
        dense_layers:Neurons in the dense layer(default [128])
        epochs:Number of epochs to tun the model (default 10)
        batch_size: Batch size (default 256)
        augment_data:Augment data or not (default 'no')
        dropout: Dropout rate (default 0.2)
    """
    def __init__(self, augment_data='no', batch_size=256, dense_layers=[128], dropout=0.2, epochs=10, layers_unfreeze=20, model="InceptionResNetV2"):
      self.augment_data = augment_data
      self.batch_size = batch_size
      self.dense_layers = dense_layers
      self.dropout = dropout
      self.epochs = epochs
      self.layers_unfreeze = layers_unfreeze
      self.model = model


In [ ]:
def generateData(augment_data_str='no', batch_size = 64):
  """
  Function to generate the train and validation dataset based on user parameters

  Attributes:
      augment_data_str: Augment data or not (default 'no')
      batch_size: Batch size (default 64)
  """

  if augment_data_str =='no':
    print("Not augmenting data")
    augment_data = False
  else:
    print("Augmenting data")
    augment_data = True

  #Generate the train and validation data
  #Validation split is 10% of train data
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,validation_split=0.1)
  IMG_SIZE = (256,256)

  if augment_data==True:  
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,rotation_range=90, shear_range=0.2, height_shift_range=0.2,
                                          width_shift_range=0.2,
                                          horizontal_flip=True,
                                          zoom_range=0.2,validation_split=0.1)

  else:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,validation_split=0.1)


  train_gen = train_datagen.flow_from_directory(
          '/content/inaturalist_12K/train',
          target_size=IMG_SIZE,
          subset = 'training',
              batch_size=batch_size,
              class_mode='categorical',
              shuffle = True,
          seed = 137)
  print('TRAINING')
  print('Number of samples', train_gen.samples)
  print('Names of classes', train_gen.class_indices)
  print('Number of classes', len(train_gen.class_indices))
  print('Number of samples per class', int(train_gen.samples / len(train_gen.class_indices) ))

  validation_gen = train_datagen.flow_from_directory(
          '/content/inaturalist_12K/train',
          target_size=IMG_SIZE,
              subset = 'validation',
              batch_size=batch_size,
              class_mode='categorical',
              shuffle = False,
          seed = 137)
  print('VALIDATION')
  print('Number of samples', validation_gen.samples)
  print('Names of classes', validation_gen.class_indices)
  print('Number of classes', len(validation_gen.class_indices))
  print('Number of samples per class', int(validation_gen.samples / len(validation_gen.class_indices) ))



  train_generator = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_types = (tf.float32, tf.float32)
    ,output_shapes = ([None, 256, 256, 3], [None, 10]),
  )
  train_generator = train_generator.repeat()
  train_generator = train_generator.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  validation_generator = tf.data.Dataset.from_generator(
      lambda: validation_gen,
      output_types = (tf.float32, tf.float32)
      ,output_shapes = ([None, 256, 256, 3], [None, 10]),
  )
  validation_generator = validation_generator.repeat()
  validation_generator = validation_generator.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_gen, train_generator, validation_gen, validation_generator


In [ ]:
def executePretrainedModel(parameters, log = False):
  """
  Function to create and execute the model based on user parameters

  Attributes:
      parameters: Object holding the parameter values
      log : Log metrics onto wandb or not (default False)
  """
  IMG_SIZE = (256,256)
  IMG_SHAPE = IMG_SIZE + (3,)

  train_gen, train_generator, validation_gen, validation_generator = generateData(parameters.augment_data, parameters.batch_size )

  #######################################################Set the model (pretrained + custom)##########################################################################

  input_ = tf.keras.Input(shape = IMG_SHAPE)

  # add a pretrained model without the top dense layer
  if parameters.model == 'ResNet50':
    pretrained_model = tf.keras.applications.ResNet50(include_top = False, weights='imagenet',input_tensor = input_)
  elif parameters.model == 'InceptionV3':
    pretrained_model = tf.keras.applications.InceptionV3(include_top = False, weights='imagenet',input_tensor = input_)
  elif parameters.model == 'InceptionResNetV2':
    pretrained_model = tf.keras.applications.InceptionResNetV2(include_top = False, weights='imagenet',input_tensor = input_)
  elif parameters.model == 'Xception':
    pretrained_model = tf.keras.applications.Xception(include_top = False, weights='imagenet',input_tensor = input_)
  elif parameters.model == 'MobileNetV2':
    pretrained_model = tf.keras.applications.MobileNetV2(include_top = False, weights='imagenet',input_tensor = input_)

  #freeze all layers
  for layer in pretrained_model.layers:
      layer.trainable=False 

  #Make last "x" layer trainable from pretrained model
  if parameters.layers_unfreeze is not None:   
    print("Training last ",parameters.layers_unfreeze," layers")
    for layer in pretrained_model.layers[-parameters.layers_unfreeze:]:
      layer.trainable=True

  model = tf.keras.models.Sequential()
  #Add pretrained model
  model.add(pretrained_model)

  #Add layers to flatten above outputs
  model.add(GlobalAveragePooling2D())
  #Add dropout
  model.add(Dropout(parameters.dropout)) 

  #Add the dense layers as passed by user
  for i in parameters.dense_layers:
    model.add(Dense(i, activation='relu'))#add a dense layer
    model.add(Dropout(parameters.dropout)) # For dropout
  model.add(Dense(10, activation="softmax"))#softmax layer

  model.summary()

  ###########################################################################################################################

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

  #Early stopping added to the model
  earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')


  #number of epochs for pre training 
  hist = model.fit(train_generator,
              epochs=parameters.epochs,
              batch_size=parameters.batch_size,
              steps_per_epoch = train_gen.samples//train_gen.batch_size,
              validation_data= validation_generator,
              validation_steps= validation_gen.samples//validation_gen.batch_size,
              callbacks=[earlyStopping]
              )
  
  val_acc=max(hist.history['val_accuracy'])
  train_acc=max(hist.history['accuracy'])
  val_loss=max(hist.history['val_loss'])
  train_loss=max(hist.history['loss']) 

  log_values = {'Training_loss':train_loss, 'Validation_loss':val_loss, 'Training_accuracy':train_acc, 'Validation_accuracy':val_acc}

  #Log metrics onto wandb if required
  if log==True:
    wandb.log(log_values)
    wandb.log({"accuracy": val_acc})

  print(log_values)


  return model


##Implementing Sweeps

The hyperparameters choosen for the sweeps are as follows:

- Model (model) : ['Xception', 'ResNet50', 'InceptionV3', 'InceptionresNetV2', 'MobileNetV2'] 
- Number of neurons in Dense Layers (dense_layers): [[256], [128], [64]]
- Epochs (epochs) : [5, 10]
- Batch Size (batch_size) : [64, 128, 256]
- Augment data (augment_data) : ['yes', 'no']
- Dropout value (dropout) : [0.2, 0.3, 0.4]
- Batch Normalization (batch_normalization) : ['yes', 'no']
- Layers to be trained in the pretrained model (layers_unfreeze) : [0, 20, 30]


1. Sweep for Xception model

In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model':{
            'values':['Xception']
        },
        'dense_layers': {
            'values': [[256], [128], [64]]
        },
        'layers_unfreeze':{
            'values': [None, 20, 30]       
        },
        'dropout': {
            'values':[0, 0.2, 0.3]
        },
        'augment_data': {
            'values': ['yes', 'no']
        },
        'batch_size' : {
            'values' : [128, 64, 256]
        },
        'epochs' : {
            'values' : [5,10]
        }

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config,project="Assignment_2", entity="hithesh-sidhesh" )

Create sweep with ID: zm216czw
Sweep URL: https://wandb.ai/hithesh-sidhesh/Assignment_2/sweeps/zm216czw


In [ ]:
#Reference link for transfer learning using a pre-trained model :- https://developer.ridgerun.com/wiki/index.php?title=Keras_with_MobilenetV2_for_Deep_Learning
#                                                                  https://keras.io/guides/transfer_learning/ 
def train():

  config_defaults = {
        'model': 'InceptionResNetV2',
        'dense_layers':[128] ,
        'layers_unfreeze':20 ,
        'augment_data' : 'no',
        'dropout' : 0.2,
        'batch_size' : 32,
        'epochs' : 10
  }
    
  with wandb.init(project='Assignment_2',config = sweep_config, name="Part B"):
    config = wandb.init().config

    #Setting custom run name 
    wandb.run.name = 'model_'+config.model + '_dense_layers_' + str(config.dense_layers) +  '_layers_unfreeze_' + str(config.layers_unfreeze) +'_augment_data_' + config.augment_data + '_dropout_' + str(config.dropout) + '_epochs_' + str(config.epochs) + '_batch_size_' + str(config.batch_size) 
    
    model = executePretrainedModel(config, log = True)
    


In [ ]:
#################################
# Setting up wandb sweeps
#################################
wandb.agent(sweep_id, train, count = 15)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: q7is9hg5 with config:
wandb: 	augment_data: no
wandb: 	batch_size: 256
wandb: 	dense_layers: [64]
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 20
wandb: 	model: Xception
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Not augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Training last  20  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)      

wandb: Agent Starting Run: wzdpmujz with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 256
wandb: 	dense_layers: [64]
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 30
wandb: 	model: Xception
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Training last  30  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)          

wandb: Agent Starting Run: 0f0shubq with config:
wandb: 	augment_data: no
wandb: 	batch_size: 128
wandb: 	dense_layers: [256]
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	layers_unfreeze: None
wandb: 	model: Xception
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Not augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                

wandb: Agent Starting Run: pcsjdeae with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 256
wandb: 	dense_layers: [128]
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	layers_unfreeze: None
wandb: 	model: Xception
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                    

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: l95bx1by with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 256
wandb: 	dense_layers: [64]
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 30
wandb: 	model: Xception
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Training last  30  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)          

2. Sweep for InceptionV3 model



In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model':{
            'values':['InceptionV3']
        },
        'dense_layers': {
            'values': [[256], [128], [64]]
        },
        'layers_unfreeze':{
            'values': [None, 20, 30]       
        },
        'dropout': {
            'values':[0, 0.2, 0.3]
        },
        'augment_data': {
            'values': ['yes', 'no']
        },
        'batch_size' : {
            'values' : [128, 64, 256]
        },
        'epochs' : {
            'values' : [5,10]
        }

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config,project="Assignment_2", entity="hithesh-sidhesh" )

Create sweep with ID: 1m08gyq0
Sweep URL: https://wandb.ai/hithesh-sidhesh/Assignment_2/sweeps/1m08gyq0


In [ ]:
#################################
# Setting up wandb sweeps
#################################
wandb.agent(sweep_id, train, count = 15)


wandb: Agent Starting Run: llj5ch8w with config:
wandb: 	augment_data: no
wandb: 	batch_size: 128
wandb: 	dense_layers: [64]
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 20
wandb: 	model: InceptionV3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Not augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
87924736/87910968 [==============================] - 1s 0us/step
Training last  20  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: scfokqza with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 64
wandb: 	dense_layers: [64]
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 30
wandb: 	model: InceptionV3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Training last  30  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)          

wandb: Agent Starting Run: gixz486v with config:
wandb: 	augment_data: no
wandb: 	batch_size: 128
wandb: 	dense_layers: [256]
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 30
wandb: 	model: InceptionV3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: Agent Starting Run: tcs3qgtv with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 128
wandb: 	dense_layers: [64]
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 20
wandb: 	model: InceptionV3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Training last  20  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)          

wandb: Agent Starting Run: 160uipp6 with config:
wandb: 	augment_data: no
wandb: 	batch_size: 64
wandb: 	dense_layers: [128]
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 20
wandb: 	model: InceptionV3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Not augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Training last  20  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)      

wandb: Ctrl + C detected. Stopping sweep.


3. Sweep for InceptionResNetV2 model



In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model':{
            'values':['InceptionResNetV2']
        },
        'dense_layers': {
            'values': [[256], [128], [64]]
        },
        'layers_unfreeze':{
            'values': [None, 20, 30]       
        },
        'dropout': {
            'values':[0, 0.2, 0.3]
        },
        'augment_data': {
            'values': ['yes', 'no']
        },
        'batch_size' : {
            'values' : [128, 64, 256]
        },
        'epochs' : {
            'values' : [5,10]
        }

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config,project="Assignment_2", entity="hithesh-sidhesh" )

Create sweep with ID: 1m08gyq0
Sweep URL: https://wandb.ai/hithesh-sidhesh/Assignment_2/sweeps/1m08gyq0


In [ ]:
#################################
# Setting up wandb sweeps
#################################
wandb.agent(sweep_id, train, count = 15)


4. Sweep for MobileNetV2 model



In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model':{
            'values':['MobileNetV2']
        },
        'dense_layers': {
            'values': [[256], [128], [64]]
        },
        'layers_unfreeze':{
            'values': [None, 20, 30]       
        },
        'dropout': {
            'values':[0, 0.2, 0.3]
        },
        'augment_data': {
            'values': ['yes', 'no']
        },
        'batch_size' : {
            'values' : [128, 64, 256]
        },
        'epochs' : {
            'values' : [5,10]
        }

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config,project="Assignment_2", entity="hithesh-sidhesh" )

Create sweep with ID: o2lidw9q
Sweep URL: https://wandb.ai/hithesh-sidhesh/Assignment_2/sweeps/o2lidw9q


In [ ]:
#################################
# Setting up wandb sweeps
#################################
wandb.agent(sweep_id, train, count = 15)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 3a5jyvwe with config:
wandb: 	augment_data: no
wandb: 	batch_size: 64
wandb: 	dense_layers: [256]
wandb: 	dropout: 0
wandb: 	epochs: 10
wandb: 	layers_unfreeze: 30
wandb: 	model: MobileNetV2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Not augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
9420800/9406464 [==============================] - 0s 0us/step
Training last  30  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                

wandb: Agent Starting Run: ws9er9n0 with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 64
wandb: 	dense_layers: [128]
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 20
wandb: 	model: MobileNetV2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Training last  20  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (No

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 52f4rbtc with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 128
wandb: 	dense_layers: [64]
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	layers_unfreeze: None
wandb: 	model: MobileNetV2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0   

wandb: Agent Starting Run: 35gdokng with config:
wandb: 	augment_data: no
wandb: 	batch_size: 256
wandb: 	dense_layers: [64]
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	layers_unfreeze: 30
wandb: 	model: MobileNetV2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Not augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Training last  30  layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G 

wandb: Agent Starting Run: 2rjer3y4 with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 256
wandb: 	dense_layers: [128]
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	layers_unfreeze: None
wandb: 	model: MobileNetV2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0   

5. Sweep for ResNet50 model



In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model':{
            'values':['ResNet50']
        },
        'dense_layers': {
            'values': [[256], [128], [64]]
        },
        'layers_unfreeze':{
            'values': [None, 20, 30]       
        },
        'dropout': {
            'values':[0, 0.2, 0.3]
        },
        'augment_data': {
            'values': ['yes', 'no']
        },
        'batch_size' : {
            'values' : [128, 64, 256]
        },
        'epochs' : {
            'values' : [5,10]
        }

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config,project="Assignment_2", entity="hithesh-sidhesh" )

Create sweep with ID: 1m08gyq0
Sweep URL: https://wandb.ai/hithesh-sidhesh/Assignment_2/sweeps/1m08gyq0


In [ ]:
#################################
# Setting up wandb sweeps
#################################
wandb.agent(sweep_id, train, count = 15)


##Best Model after Transfer learning

The hyperparameters of the best model are as follows:

- augment_data: no
- batch_size: 256
- dense_layers: [128]
- dropout: 0.2
- epochs: 5
- layers_unfreeze: 20
- model: InceptionResNetV2

In [ ]:
parameters = ConfigValues(augment_data='no', batch_size=256, dense_layers=[128], dropout=0.2, epochs=5, layers_unfreeze=20, model="InceptionResNetV2")

In [ ]:
model = executePretrainedModel(parameters, log = False)

Not augmenting data
Found 9000 images belonging to 10 classes.
TRAINING
Number of samples 9000
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 900
Found 999 images belonging to 10 classes.
VALIDATION
Number of samples 999
Names of classes {'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}
Number of classes 10
Number of samples per class 99
Training last  20  layers
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 6, 6, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_3

In [ ]:
  test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
  test_gen = test_datagen.flow_from_directory(
          '/content/inaturalist_12K/val',
          target_size=(256,256),
              batch_size=parameters.batch_size,
              class_mode='categorical',
              shuffle = False,
          seed = 137)
  test_generator = tf.data.Dataset.from_generator(
      lambda: test_gen,
      output_types = (tf.float32, tf.float32)
      ,output_shapes = ([None, 256, 256, 3], [None, 10]),
  )
  test_generator = test_generator.repeat()
  test_generator = test_generator.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  test_loss, test_acc = model.evaluate(test_generator, steps=test_gen.samples//test_gen.batch_size, verbose=2)
  print('Test accuracy :', test_acc)

Found 2000 images belonging to 10 classes.
7/7 - 19s - loss: 0.5480 - accuracy: 0.8298 - 19s/epoch - 3s/step
Test accuracy : 0.8297991156578064


##Results from Best model:

###Accuracy score:
- Training Accuracy: 88.54%
- Validation Accuracy: 82.55%
- Test Accuracy: 82.97%

###Loss values:
- Training Loss: 0.3559
- Validation Loss: 0.5786
- Test Loss: 0.5480

##Save model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


from keras.models import load_model
from google.colab import files
MODEL_PATH = '/content/best_model_pretrained.h5'

#model.save(MODEL_PATH)
# Load Model
model = load_model(MODEL_PATH)


Mounted at /content/drive
